In [ ]:
train = ['population genomics of saccharomyces cerevisiae human isolates: passengers, colonizers, invaders.',
 'estimating seed bank accumulation and dynamics in three obligateseeder proteaceae species',
 'how and where to look for trnas in metazoan mitochondrial genomes, and what you might find when you get there',
 'tracking global changes induced in the cd4 t cell receptor repertoire by immunization with a complex antigen using short stretches of cdr3 protein sequence.',
 'the shrinking human protein coding complement: are there fewer than 20,000 genes?',
 'emergence of structural and dynamical properties of ecological mutualistic networks',
 'expertly validated models suggest responses to climate change are related to species traits: a phylogeneticallycontrolled analysis of the order lagomorpha',
 'the emergence of the rescue effect from explicit within and betweenpatch dynamics in a metapopulation',
 'the toxoplasma actomyoa motor complex is important but not essential for gliding motility and host cell invasion',
 'human paternal and maternal demographic histories: insights from highresolution y chromosome and mtdna sequences']

In [99]:
def split_len(splits):
    total = 0
    for s in splits:
        total += len(s)
    return total
def init_vocab(splits):
    vocab = set()
    for s in splits:
        for t in s:
            vocab.add(t)
    return list(vocab)

def compute_pair_frequencies(splits: list[list[str]]):
    counts = {}
    for s in splits:
        for i in range(len(s)-1):
            k = (s[i], s[i+1])
            if k in counts:
                counts[k] += 1
            else:
                counts[k] = 1
    return [(counts[k], k) for k in counts]


def max_freq(freqs: list[tuple[int, str]]):
    max_count, max_str = freqs[0]
    for i in range(1, len(freqs)):
        c, s = freqs[i]
        if c > max_count:
            max_count = c
            max_str = s
    return max_count, max_str

def merge_pair_all(split, pair, pair_concat):
    result = []
    i = 0
    while i < len(split):
        if i < (len(split)-1) and (pair[0] == split[i] and pair[1] == split[i+1]):
            result.append(pair_concat)
            i+=1
        else:
            result.append(split[i])
        i+=1
    return result

class BPE:
    def __init__(self, train, n_vocab=256):
        self.train = train
        self.n_vocab = n_vocab
    
    def split_train(self):
        res = []
        for t in train:
            sub = []
            for c in t:
                sub.append(c)
            res.append(sub)
        return res

    def compute_bpe(self):
        splits = self.split_train()
        vocab = init_vocab(splits)
        merges = []

        while len(vocab) < self.n_vocab:
            # count pairs
            counts = compute_pair_frequencies(splits)

            # find most frequent pair
            freq, pair = max_freq(counts)

            # add to history
            vocab_entry = pair[0] + pair[1] # concat the string
            merges.append((pair, vocab_entry))
            vocab.append(vocab_entry)

            for i in range(len(splits)):
                splits[i] = merge_pair_all(splits[i], pair, vocab_entry)
        
        # save and return
        self.vocab = vocab
        self.merges = merges
        return self
    
    def encode(self, x: str):
        assert self.vocab and self.merges, "First .compute_bpe() or load_bpe()"
        split = list(x)
        for m in self.merges:
            split = merge_pair_all(split, m[0], m[1])
        return split

        

bpe = BPE(train, n_vocab=128).compute_bpe()
for t in train: 
    e = bpe.encode(t)
    print(len(e), len(t))
    print(''.join(e))
    print(''.join(e) == t)

37 97
population genomics of saccharomyces cerevisiae human isolates: passengers, colonizers, invaders.
True
40 89
estimating seed bank accumulation and dynamics in three obligateseeder proteaceae species
True
60 109
how and where to look for trnas in metazoan mitochondrial genomes, and what you might find when you get there
True
89 156
tracking global changes induced in the cd4 t cell receptor repertoire by immunization with a complex antigen using short stretches of cdr3 protein sequence.
True
42 81
the shrinking human protein coding complement: are there fewer than 20,000 genes?
True
42 83
emergence of structural and dynamical properties of ecological mutualistic networks
True
89 154
expertly validated models suggest responses to climate change are related to species traits: a phylogeneticallycontrolled analysis of the order lagomorpha
True
47 101
the emergence of the rescue effect from explicit within and betweenpatch dynamics in a metapopulation
True
64 112
the toxoplasma actomyoa